<a href="https://colab.research.google.com/github/dasom222g/learn-LLM/blob/main/02_1_text_basic_preprocessing(%E1%84%8B%E1%85%A7%E1%86%BC%E1%84%8B%E1%85%A5%E1%84%86%E1%85%B5%E1%86%BE%E1%84%92%E1%85%A1%E1%86%AB%E1%84%80%E1%85%AE%E1%86%A8%E1%84%8B%E1%85%A5).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#텍스트 전처리(Text Preprocessing)

- 텍스트를 자연어 처리를 위해 용도에 맞도록 사전에 표준화 하는 작업
- 텍스트 내 정보를 유지하고, 중복을 제거하여 분석 효율성을 높이기 위해 전처리를 수행

1) 토큰화(Tokenizing)
- 텍스트를 자연어 처리를 위해 분리하는 것
- 토큰화는 단어별로 분리하는 "단어 토큰화(Word Tokenization)"과 문장별로 분리하는 "문장 토큰화(Sentence Tokenization)"으로 구분  
=> 이후 실습에서는 단어 토큰화를 "토큰화"로 통일하여 칭함

2) 노이즈 및 불용어 처리
 - 자연어 처리를 위해 불필요한 요소를 제거하는 작업
 - 불필요한 품사를 제거하는 작업과 불필요한 단어를 제거하는 작업으로 구성
 - 불필요한 토큰을 제거함으로써 연산의 효율성을 높임

3) 원형복원(Stemming & Lemmatization)
- 각 토큰의 원형을 복원함으로써 토큰을 표준화 하여 불필요한 데이터 중복을 방지  
(= 단어의 수를 줄일 수 있어 연산 효율성을 높임)  

 - 어간추출(Stemming): 품사를 무시하고 규칙에 기반하여 어간을 추출
 - 표제어 추출(Lemmatization): 품사정보를 유지하여 표제어 추출

4) 품사 부착(Pos Tagging)
- 각 토큰에 품사 정보를 추가
- 분석시에 불필요한 품사를 제거하거나(예. 조사, 접속사 등) 필요한 품사를 필터링 하기 위해 사용

5) 개체명 인식(NER, Named Entity Recognition)
- 각 토큰의 개체구분(기관, 인물, 지역, 날짜 등) 태그를 부착
- 텍스트가 무엇과 관련되어 있는지 구분하기 위해 사용
- 예를 들어, 과일의 apple과 기업의 apple을 구분하는 방법이 개체명 인식임.




#1. 영문 전처리 연습

NLTK lib(https://www.nltk.org/)사용

## 실습용 영문기사 수집

온라인 기사를 바로 수집하여 실습데이터로 사용
https://www.forbes.com/sites/adrianbridgwater/2019/04/15/what-drove-the-ai-renaissance/#4a0130481f25

In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
url = 'https://www.forbes.com/sites/adrianbridgwater/2019/04/15/what-drove-the-ai-renaissance/#4a0130481f25'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

In [ ]:
eng_news = soup.select('p') #[class = 'speakable-paragraph']
eng_text = eng_news[3].get_text()

eng_text

', Senior Contributor. '

## 1) 영문토큰화
https://www.nltk.org/api/nltk.tokenize.html

In [ ]:
# 영문 자연어 처리 및 문서 분석용 파이썬 패키지

!pip install nltk

In [ ]:
#word_tokenize(): 단어와 구두점(온점, 컴마, 물음표, 세미콜론, 느낌표 등과 같은 기호)으로 단어 토큰화
#LineTokenizer(): 줄바꿈 문자('\n')를 기준으로 토큰화
#sent_tokenize(): 문장 기준으로 토큰화
#SpaceTokenizer(): 공백 기준으로 토큰화

import nltk
# nltk.download('punkt')
nltk.download('punkt_tab') # 말뭉치 다운로드
nltk.download('omw-1.4')

from nltk.tokenize import word_tokenize
import os

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [ ]:
text = "James is working at Disney in London"
word_tokens = word_tokenize(text)
print(word_tokens)

['James', 'is', 'working', 'at', 'Disney', 'in', 'London']


In [ ]:
#WordPunctTokenizer(): 알파벳과 알파벳이 아닌 문자를 구분하여 토큰화
import nltk
from nltk.tokenize import WordPunctTokenizer

text =  "James is working at Disney in London"'
wordpuncttoken = WordPunctTokenizer().tokenize(text)
print(wordpuncttoken)

['James', 'is', 'working', 'at', 'Disney', 'in', 'London']


In [ ]:
#TreebankWordTokenizer(): 정규표현식에 기반한 토큰화
import nltk
from nltk.tokenize import TreebankWordTokenizer

text =  "James is working at Disney in London"
treebankwordtoken = TreebankWordTokenizer().tokenize(text)
print(treebankwordtoken)

['James', 'is', 'working', 'at', 'Disney', 'in', 'London']


## 2) 노이즈 및 불용어(Stopword) 처리

#### 노이즈 처리 기준

- 특수 문자 및 기호  
    HTML/XML 태그 (<div>, <p> 등)  
    구두점 (.,!?:;)  
    괄호, 대시, 슬래시 등 (()[]{}-/)  
    이모티콘과 이모지 (😊, :), :-P)  

- 숫자와 관련 표현  
    일반 숫자 (1, 2, 3...)  
    날짜 형식 (2023-03-07, 07/03/2023)  
    금액 표시 ($100, ₩10,000)  

- 잘못된 입력 및 오류  
    오타 및 맞춤법 오류  
    인코딩 오류 (깨진 문자)  

- 포맷 관련 요소  
    URL (http://example.com)  
    이메일 주소 (user@example.com)  
    파일 경로 (/usr/local/bin)  
    사용자 ID, 해시태그 (@user, #hashtag)  

- 저빈도 출현 요소  
    코퍼스에서 1-5회 미만 등장하는 단어  
    특정 문서에만 등장하는 희귀 단어
  
⭐️ 코퍼스: 말뭉치로 전체 토큰 목록을 의미

#### 불용어에 해당하는 것들

- 일반적 고빈도 동사  
    be 동사 (am, is, are, was, were)  
    have/has/had  
    do/does/did  

- 분석 목적에 따른 중립(일반적인) 단어  
    감성분석에서 중립적인 표현 단어  
    주제 분류에 영향이 적은 일반 단어  

- 메타 단어  
    교과서 형식의 텍스트에서 "chapter", "figure"  
    뉴스에서 "reported", "according"  
    학술 논문에서 "study", "research", "analysis"  

- 고빈도 단어  
    코퍼스의 상위 1-5%를 차지하는 단어  
    대부분의 문서(80% 이상)에 등장하는 단어  
    TF-IDF 값이 매우 낮은 단어  

- 특정 언어별 불용어  
    한국어: 이, 그, 저, 것, 등, 및, 에서, 으로  
    영어: this, that, these, those, such, what  
    일본어: これ, それ, あれ, この, その  

- 문법적 기능어  
    관사 (a, an, the)  
    전치사 (in, on, at, by, with)  
    접속사 (and, but, or, so)  
    조동사 (can, will, should)  
    대명사 (I, you, he, she, they)

In [ ]:
stop_words = ["at", "be", "able", "in"]

In [ ]:
word_tokens = []
for tag in treebankwordtoken:
    if len(tag) > 1:
        if tag not in stop_words:
            word_tokens.append(tag)

print(word_tokens)

['James', 'is', 'working', 'Disney', 'London']


In [ ]:
# 최빈어 조회: 최빈어를 조회하여 불용어 제거
from collections import Counter
Counter(word_tokens).most_common()

[('James', 1),
 ('is', 1),
 ('working', 1),
 ('at', 1),
 ('Disney', 1),
 ('in', 1),
 ('London', 1)]

### 문제
word_tokens에서 빈도수가 5개 이상인 토큰은 삭제

In [ ]:
text = 'The cat sat on the mat while the dog slept by the door. The cat noticed the bird on the tree and watched it carefully. The dog woke up when the mailman arrived at the door. The children played with the cat and the dog in the garden all afternoon.'
word_tokens = nltk.word_tokenize(text)

result = []

# 빈도수 조회
token_numbers = Counter(word_tokens).most_common()

for word, num in token_numbers:
  # print(word)
  # print(num)
  # print('='*30)
  if num < 5:
    result.append(word)

result

['The',
 '.',
 'cat',
 'dog',
 'on',
 'door',
 'and',
 'sat',
 'mat',
 'while',
 'slept',
 'by',
 'noticed',
 'bird',
 'tree',
 'watched',
 'it',
 'carefully',
 'woke',
 'up',
 'when',
 'mailman',
 'arrived',
 'at',
 'children',
 'played',
 'with',
 'in',
 'garden',
 'all',
 'afternoon']

## 3) 원형복원
각 토큰의 원형을 복원하여 표준화한다

### 3-1) 어간추출(Stemming)
- 규칙에 기반하여 토큰을 표준화
- ing제거, ful제거 등  
http://www.nltk.org/api/nltk.chunk.html

규칙 상세: http://www.nltk.org/api/nltk.chunk.html


In [ ]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()

In [ ]:
print("running -> " + ps.stem("running"))
print("believes -> "+ps.stem('believes'))
print('using ->' + ps.stem("using"))
print("conversation ->" + ps.stem('conversation'))
print('organization ->'+ ps.stem('organization'))
print('studies -> '+ ps.stem("studies"))

running -> run
believes -> believ
using ->use
conversation ->convers
organization ->organ
studies -> studi


### 3-2) 표제어 추출(Lemmatization)
- 품사정보를 유지하여 토큰을 표준화
- 어간 추출보다 완성형 토큰 보관가능

http://www.nltk.org/api/nltk.chunk.html

In [ ]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
from nltk.stem import WordNetLemmatizer
wl = WordNetLemmatizer()

In [ ]:
print("running -> " + wl.lemmatize("running"))
print("believes -> "+wl.lemmatize('believes'))
print('using ->' + wl.lemmatize("using"))
print("conversation ->" + wl.lemmatize('conversation'))
print('organization ->'+ wl.lemmatize('organization'))
print('studies -> '+ wl.lemmatize("studies"))

running -> running
believes -> belief
using ->using
conversation ->conversation
organization ->organization
studies -> study


#### 문제
어간추출을 이용하여 원형복원해보세요.

In [ ]:
word_tokens

['James', 'is', 'working', 'Disney', 'London']

In [ ]:
lemmatizer_word_tokens = [wl.lemmatize(word) for word in word_tokens]
lemmatizer_word_tokens

['James', 'is', 'working', 'Disney', 'London']

## 4) 영문 품사 부착(PoS tagging)

분리한 토큰마다 품사를 부착한다.   
https://www.nltk.org/api/nltk.tag.html

태그목록: https://pythonprogramming.net/natural-language-toolkit-nltk-part-speech-tagging/

## 자주 사용되는 품사종류
- NN: 명사, 단수형 또는 불가산
- NNS: 명사, 복수형
- NNP: 고유 명사, 단수형
- VB: 동사, 원형
- VBD: 동사, 과거형
- VBG: 동사, 현재분사/동명사
- VBN: 동사, 과거분사
- VBZ: 동사, 3인칭 단수 현재
- JJ: 형용사
- RB: 부사
- IN: 전치사 또는 종속접속사
- DT: 한정사(관사 등)
- CC: 등위접속사
- PRP: 인칭 대명사
- TO: to (전치사/부정사 표지)
- MD: 조동사

In [ ]:
from nltk import pos_tag
nltk.download('averaged_perceptron_tagger_eng') # 품사 맵핑의 기준이 되는 데이터

[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


True

In [ ]:
tagged_tokens = pos_tag(word_tokens)
print(tagged_tokens)

[('James', 'NNP'), ('is', 'VBZ'), ('working', 'VBG'), ('Disney', 'NNP'), ('London', 'NNP')]


In [ ]:
# 불필요한 품사 제거
stop_pos = ["IN", "CC", "UH", "TO", "MD", "DT", "VBZ", "VBP"]


#### 문제
stop_pos에 해당하는 제거해보세요

In [ ]:
# stop_pos에 포함되지 않는 품사만 추출
result =  [(token, part) for token, part in tagged_tokens if part not in stop_pos]
result

[('James', 'NNP'), ('working', 'VBG'), ('Disney', 'NNP'), ('London', 'NNP')]

##5) 개체명 인식(NER, Named Entity Recognition)
http://www.nltk.org/api/nltk.chunk.html


개체명 인식(Named Entity Recognition)이란 말 그대로 이름을 가진 개체(named entity)를 인식하겠다는 것을 의미합니다.   
즉, 어떤 이름을 의미하는 단어를 보고는 그 단어가 어떤 유형인지를 인식하는 것을 말합니다.

예를 들어 제임스는 2018년에 골드만삭스에 입사했다. 라는 문장이 있을 때, 사람(person), 조직(organization), 시간(time)에 대해 개체명 인식을 수행하는 모델이라면 다음과 같은 결과를 보여줍니다.

- 제임스 : 사람
- 2018년 : 시간
- 골드만삭스 : 조직

In [ ]:
nltk.download('words') # 영어 단어 목록을 다운로드 (참조 데이터로 활용)
nltk.download('maxent_ne_chunker_tab') # 개체명 인식(NER)을 위한 최대 엔트로피 기반 모델 다운로드 (개체명 식별)

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package maxent_ne_chunker_tab to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker_tab.zip.


True

In [ ]:
tagged_tokens

[('James', 'NNP'),
 ('is', 'VBZ'),
 ('working', 'VBG'),
 ('Disney', 'NNP'),
 ('London', 'NNP')]

In [ ]:
# 개체명 인식해주는 라이브러리
from nltk import ne_chunk
ne_tokens = ne_chunk(tagged_tokens)
print(ne_tokens)

(S
  (PERSON James/NNP)
  is/VBZ
  working/VBG
  (PERSON Disney/NNP London/NNP))


James는 PERSON(사람), Disney는 조직(ORGANIZATION), London은 위치(GPE)

#2. 한글 전처리 실습

영문은 공백으로 토큰화가 가능하지만, 한글의 경우 품사를 고려하여 토큰화해야 한다.

- 한글 토큰화 단위: 형태소 - 의미를 가진 최소단위

## 실습용 한글기사 수집


- https://n.news.naver.com/mnews/article/016/0002032876?sid=105

In [ ]:
kor_text = '''포스코ICT는 25일 경기도 판교 사옥에서 청소년들이 개발한 인공지능(AI) 기반 애플리케이션을 직접 선보이는 ‘2022 AI 유스 챌린지(Youth Challenge)’를 개최했다. 청소년들은 기성세대들이 생각하지 못했던 참신한 아이디어를 바탕으로 AI 시스템을 개발해 관심을 모았다.

예선을 거쳐 본선에 오른 전국 중·고교 7개팀은 지난 달부터 포스코ICT 소속 AI 엔지니어의 코칭 속에 개발한 AI 시스템을 직접 시연했다. 심사는 카이스트, 포스텍, 연세대 교수 등으로 구성된 평가위원들과 포스코ICT 직원들로 구성된 내부 평가위원들이 맡았다. 우수작은 과기정통부장관상(1팀), 포스코ICT사장상(1팀), 지능정보산업협회장상(1팀), 우수상(2팀), 장려상(2팀) 등으로 나눠 각각 시상했다. 포스코ICT는 AI 기술을 접목해 환경·안전·사회문제를 해결할 수 있는 청소년들의 아이디어를 발굴하기 위해 이번 공모전을 개최했다. 평소 AI 기술을 접할 기회가 부족했던 청소년들은 멘토링을 통해 산업현장에서 실제 활용되는 기술을 자연스럽게 체험하고 진로 결정에도 도움을 받을 수 있었다.

이번 공모전에 참여한 김희주 학생은 “막연히 생각했던 아이디어를 실제 AI 전문가들과 구체화해나가며 접하지 못했던 지식과 기술을 배울 수 있었고 AI 분야에 더 큰 관심을 가지게 됐다”고 말했다.

학생들의 멘토를 맡은 포스코ICT 최영철 연구원은 “학생들의 열정과 창의적인 아이디어를 접하며 새로운 자극을 받아 재미있게 멘토링에 참여했다”며 “그동안 AI 분야에서 쌓아온 재능을 기부해 AI 전문가를 꿈꾸는 청소년들에게 전해줄 수 있어 의미 있었다”고 밝혔다. 한편, 포스코ICT는 아주대학교 대학원과 ‘AI 전문인력 양성을 위한 업무협약’을 체결해 프로그램을 운영하고 있다. 지난 24일에는 인공지능학과 대학원 과정에 재학 중인 우수 인재를 선발해 장학금을 지급했다.

이번에 선발된 학생에게는 졸업 때까지 매달 장학금을 지급하고, 포스코ICT의 AI 전문가와의 1대1 멘토링 및 각종 기술 교육, 세미나 참석을 비롯해 채용 기회까지 제공할 계획이다. 김현일 기자'''

## 1) 한글 토큰화 및 형태소 분석

한글 자연어 처리기 비교  
https://konlpy.org/ko/latest/morph/



#### 형태소 분석기별 메서드 정리
출처 : https://konlpy-ko.readthedocs.io/ko/v0.4.3/

---
1) Hannanum : KAIST에서 개발   
- analyze : 다양한 형태학적 후보를 반환
- morphs : 형태소 토큰화
- nouns : 명사 토큰만 반환
- pos : 형태소에 품사 태깅하여 반환
---  
2) Kkma : 서울대에서 개발
- morphs : 형태소 토큰화
- nouns : 명사 토큰만 반환
- pos : 형태소에 품사 태깅하여 반환
- sentences : 문장 토큰화
---  
3) Komoran : Shineware팀에서 개발
- morphs : 형태소 토큰화
- nouns : 명사 토큰만 반환
- pos : 형태소에 품사 태깅하여 반환
---
4) Mecab : 교토대에서 일본어용으로 개발된 분석기, 은전 프로젝트에 의해 한국어용 개발
- morphs : 형태소 토큰화
- nouns : 명사 토큰만 반환
- pos : 형태소에 품사 태깅하여 반환
---
5) Okt(Twitter) : Will Hohyon Ryu가 개발
- morphs : 형태소 토큰화
- nouns : 명사 토큰만 반환
- phrases : 다양한 형태학적 후보를 반환
- pos : 형태소에 품사 태깅하여 반환


In [ ]:
#konlpy 설치
# 한국어 자연어 전처리용 라이브러리로 형태소 분석 가능
!pip install konlpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 71.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.1/494.1 kB 27.8 MB/s eta 0:00:00


In [ ]:
#코모란(Komoran) 토큰화

from konlpy.tag import Komoran
komoran = Komoran()
kor_text = "인간이 컴퓨터와 대화하고 있다는 것을 꺠닫지 못하고 인간과 대화를 계속할 수 있다면 컴퓨터는 지능적인 것으로 간주될 수 있다."
komoran_tokens = komoran.morphs(kor_text)
print(komoran_tokens)

['인간', '이', '컴퓨터', '와', '대화', '하', '고', '있', '다는', '것', '을', '꺠닫지', '못하', '고', '인간', '과', '대화', '를', '계속', '하', 'ㄹ', '수', '있', '다면', '컴퓨터', '는', '지능', '적', '이', 'ㄴ', '것', '으로', '간주', '되', 'ㄹ', '수', '있', '다', '.']


In [ ]:
#한나눔(hannanum) 토큰화
from konlpy.tag import Hannanum
hannanum = Hannanum()
kor_text = "인간이 컴퓨터와 대화하고 있다는 것을 꺠닫지 못하고 인간과 대화를 계속할 수 있다면 컴퓨터는 지능적인 것으로 간주될 수 있다."
hannanum_tokens = hannanum.morphs(kor_text)
print(hannanum_tokens)

['인간', '이', '컴퓨터', '와', '대화', '하고', '있', '다는', '것', '을', '꺠닫지', '못하', '고', '인간', '과', '대화', '를', '계속', '하', 'ㄹ', '수', '있', '다면', '컴퓨터', '는', '지능적', '이', 'ㄴ', '것', '으로', '간주', '되', 'ㄹ', '수', '있', '다', '.']


In [ ]:
#Okt 토큰화
from konlpy.tag import Okt
okt = Okt()
okt_tokens = okt.morphs(kor_text)
print(okt_tokens)

['인간', '이', '컴퓨터', '와', '대화', '하고', '있다는', '것', '을', '꺠닫', '지', '못', '하고', '인간', '과', '대화', '를', '계속', '할', '수', '있다면', '컴퓨터', '는', '지능', '적', '인', '것', '으로', '간주', '될', '수', '있다', '.']


In [ ]:
# Kkma 토큰화
from konlpy.tag import Kkma
kkma = Kkma()
kkma_tokens = kkma.morphs(kor_text)
print(kkma_tokens)

['인간', '이', '컴퓨터', '와', '대화', '하', '고', '있', '다는', '것', '을', '꺠', '닫', '지', '못하', '고', '인간', '과', '대화', '를', '계속', '하', 'ㄹ', '수', '있', '다면', '컴퓨터', '는', '지능', '적', '이', 'ㄴ', '것', '으로', '간주', '되', 'ㄹ', '수', '있', '다', '.']


In [ ]:
kor_text = '그래욬ㅋㅋㅋ잘해봅시다! 잘하고 있지만 더 잘해보면 좋지 않을까라고 생각하는 중'

In [ ]:
from konlpy.tag import Okt
okt = Okt()

In [ ]:
print(okt.morphs(kor_text, norm = False))

['그래욬', 'ㅋㅋㅋ', '잘', '해봅시다', '!', '잘', '하고', '있지만', '더', '잘', '해보면', '좋지', '않을까', '라고', '생각', '하는', '중']


In [ ]:
print(okt.morphs(kor_text, norm=True))

['그래요', 'ㅋㅋㅋ', '잘', '해봅시다', '!', '잘', '하고', '있지만', '더', '잘', '해보면', '좋지', '않을까', '라고', '생각', '하는', '중']


In [ ]:
print(okt.morphs(kor_text, stem = True))
# 좋지 -> 좋다
# 않을까 -> 않다

['그래욬', 'ㅋㅋㅋ', '잘', '해보다', '!', '잘', '하다', '있다', '더', '잘', '해보다', '좋다', '않다', '라고', '생각', '하다', '중']


## 2) 한글 품사 부착(PoS Tagging)

Pos Tag 목록
https://docs.google.com/spreadsheets/u/1/d/1OGAjUvalBuX-oZvZ_-9tEfYD2gQe7hTGsgUpiiBSXI8/edit#gid=0

In [ ]:
# 코모란(Komoran) 품사 태깅
komoran_tags = []
for token in komoran_tokens:
  komoran_tags += komoran.pos(token)
print(komoran_tags)

[('인간', 'NNG'), ('이', 'MM'), ('컴퓨터', 'NNG'), ('오', 'VV'), ('아', 'EC'), ('대화', 'NNG'), ('하', 'NNG'), ('고', 'MM'), ('있', 'VV'), ('달', 'VV'), ('는', 'ETM'), ('것', 'NNB'), ('을', 'NNG'), ('꺠닫지', 'NA'), ('못', 'MAG'), ('하', 'MAG'), ('고', 'MM'), ('인간', 'NNG'), ('과', 'NNG'), ('대화', 'NNG'), ('를', 'JKO'), ('계속', 'MAG'), ('하', 'NNG'), ('ㄹ', 'NA'), ('수', 'NNB'), ('있', 'VV'), ('다면', 'NNG'), ('컴퓨터', 'NNG'), ('늘', 'VV'), ('ㄴ', 'ETM'), ('지능', 'NNP'), ('적', 'NNB'), ('이', 'MM'), ('ㄴ', 'JX'), ('것', 'NNB'), ('으로', 'JKB'), ('간주', 'NNG'), ('되', 'NNB'), ('ㄹ', 'NA'), ('수', 'NNB'), ('있', 'VV'), ('다', 'MAG'), ('.', 'SF')]


In [ ]:
# 한나눔(Hannanum) 품사 태깅
hannanum_tags = []
for token in hannanum_tokens:
  hannanum_tags += hannanum.pos(token)

print(hannanum_tags)

[('인간', 'N'), ('이', 'M'), ('컴퓨터', 'N'), ('와', 'I'), ('대화', 'N'), ('하', 'P'), ('고', 'E'), ('있', 'N'), ('다', 'M'), ('는', 'J'), ('것', 'N'), ('을', 'N'), ('꺠닫지', 'N'), ('못하', 'P'), ('어', 'E'), ('고', 'M'), ('인간', 'N'), ('과', 'N'), ('대화', 'N'), ('를', 'N'), ('계속', 'M'), ('하', 'I'), ('ㄹ', 'N'), ('수', 'N'), ('있', 'N'), ('다면', 'N'), ('컴퓨터', 'N'), ('늘', 'P'), ('ㄴ', 'E'), ('지능적', 'N'), ('이', 'M'), ('ㄴ', 'N'), ('것', 'N'), ('으', 'N'), ('로', 'J'), ('간주', 'N'), ('되', 'N'), ('ㄹ', 'N'), ('수', 'N'), ('있', 'N'), ('다', 'M'), ('.', 'S')]


In [ ]:
# Okt 품사 태깅
okt_tags = []
for token in okt_tokens:
  okt_tags += okt.pos(token)
print(okt_tags)

[('인간', 'Noun'), ('이', 'Noun'), ('컴퓨터', 'Noun'), ('와', 'Verb'), ('대화', 'Noun'), ('하고', 'Verb'), ('있다는', 'Adjective'), ('것', 'Noun'), ('을', 'Josa'), ('꺠닫', 'Noun'), ('지', 'Verb'), ('못', 'Noun'), ('하고', 'Verb'), ('인간', 'Noun'), ('과', 'Noun'), ('대화', 'Noun'), ('를', 'Noun'), ('계속', 'Noun'), ('할', 'Verb'), ('수', 'Noun'), ('있다면', 'Adjective'), ('컴퓨터', 'Noun'), ('는', 'Verb'), ('지능', 'Noun'), ('적', 'Noun'), ('인', 'Noun'), ('것', 'Noun'), ('으로', 'Josa'), ('간주', 'Noun'), ('될', 'Verb'), ('수', 'Noun'), ('있다', 'Adjective'), ('.', 'Punctuation')]


In [ ]:
#Kkma 품사 태깅
kkma_tags = []
for token in kkma_tokens:
  kkma_tags += kkma.pos(token)
print(kkma_tags)

[('인간', 'NNG'), ('이', 'NNG'), ('컴퓨터', 'NNG'), ('오', 'VA'), ('아', 'ECS'), ('대화', 'NNG'), ('하', 'NNG'), ('고', 'NNG'), ('있', 'VA'), ('달', 'VV'), ('는', 'ETD'), ('것', 'NNB'), ('을', 'NNG'), ('꺠', 'UN'), ('닫', 'VV'), ('지', 'NNG'), ('못하', 'VX'), ('고', 'NNG'), ('인간', 'NNG'), ('과', 'NNG'), ('대화', 'NNG'), ('를', 'UN'), ('계속', 'MAG'), ('하', 'NNG'), ('ㄹ', 'NNG'), ('수', 'NNG'), ('있', 'VA'), ('다면', 'NNG'), ('컴퓨터', 'NNG'), ('늘', 'VA'), ('ㄴ', 'ETD'), ('지능', 'NNG'), ('적', 'NNG'), ('이', 'NNG'), ('ㄴ', 'NNG'), ('것', 'NNB'), ('으', 'UN'), ('로', 'JKM'), ('간주', 'NNG'), ('되', 'VA'), ('ㄹ', 'NNG'), ('수', 'NNG'), ('있', 'VA'), ('다', 'NNG'), ('.', 'SF')]


## 3) 노이즈 및 불용어(StopWord) 처리

분석에 불필요한 품사를 제거하고, 불필요한 단어(불용어)를 제거한다.

In [ ]:
# 불용어 처리
stop_pos = ['Suffix','Punctuation','Josa','Foreign','Alpha','Number'] # 품사

In [ ]:
#최빈어 조회: 최빈어를 조회하여 불용어 제거 대상을 선정
from collections import Counter
Counter(okt_tags).most_common()

[(('인간', 'Noun'), 2),
 (('컴퓨터', 'Noun'), 2),
 (('대화', 'Noun'), 2),
 (('하고', 'Verb'), 2),
 (('것', 'Noun'), 2),
 (('수', 'Noun'), 2),
 (('이', 'Noun'), 1),
 (('와', 'Verb'), 1),
 (('있다는', 'Adjective'), 1),
 (('을', 'Josa'), 1),
 (('꺠닫', 'Noun'), 1),
 (('지', 'Verb'), 1),
 (('못', 'Noun'), 1),
 (('과', 'Noun'), 1),
 (('를', 'Noun'), 1),
 (('계속', 'Noun'), 1),
 (('할', 'Verb'), 1),
 (('있다면', 'Adjective'), 1),
 (('는', 'Verb'), 1),
 (('지능', 'Noun'), 1),
 (('적', 'Noun'), 1),
 (('인', 'Noun'), 1),
 (('으로', 'Josa'), 1),
 (('간주', 'Noun'), 1),
 (('될', 'Verb'), 1),
 (('있다', 'Adjective'), 1),
 (('.', 'Punctuation'), 1)]

In [ ]:
stop_words = ['의','와','이','로','두고','들','를','은','과','수','했다','것','있는','한다','하는','그','있다']

In [ ]:
len(okt_tags)

33

In [ ]:
word =[]
for tag in okt_tags:
  if tag[1] not in stop_pos:
    if tag[0] not in stop_words:
      word.append(tag[0])

print(word)
len(word)

['인간', '컴퓨터', '대화', '하고', '있다는', '꺠닫', '지', '못', '하고', '인간', '대화', '계속', '할', '있다면', '컴퓨터', '는', '지능', '적', '인', '간주', '될']


21

In [ ]:
result = []

for tag, pos in okt_tags:
  if pos not in stop_pos and tag not in stop_words: # 품사 체크
    result.append(tag)
print(result)
len(result)

['인간', '컴퓨터', '대화', '하고', '있다는', '꺠닫', '지', '못', '하고', '인간', '대화', '계속', '할', '있다면', '컴퓨터', '는', '지능', '적', '인', '간주', '될']


21

## 문제
kkma를 이용하여 빈도수가 3이상이거나 아래 품사에 해당하는 토큰들 제거

- ECS, VA, ETD

In [ ]:
kor_text = "인간이 컴퓨터와 대화하고 있다는 것을 꺠닫지 못하고 인간과 대화를 계속할 수 있다면 컴퓨터는 지능적인 것으로 간주될 수 있다."

# Kkma 토큰화
from konlpy.tag import Kkma

kkma = Kkma()
kkma_tokens = kkma.morphs(kor_text)
print(kkma_tokens)

['인간', '이', '컴퓨터', '와', '대화', '하', '고', '있', '다는', '것', '을', '꺠', '닫', '지', '못하', '고', '인간', '과', '대화', '를', '계속', '하', 'ㄹ', '수', '있', '다면', '컴퓨터', '는', '지능', '적', '이', 'ㄴ', '것', '으로', '간주', '되', 'ㄹ', '수', '있', '다', '.']


In [ ]:
tokens_tags = []
for token in kkma_tokens:
  tokens_tags.append(kkma.pos(token)[0])

print(tokens_tags)

stop_pos = ['ECS', 'VA', 'ETD']

[('인간', 'NNG'), ('이', 'NNG'), ('컴퓨터', 'NNG'), ('오', 'VA'), ('대화', 'NNG'), ('하', 'NNG'), ('고', 'NNG'), ('있', 'VA'), ('달', 'VV'), ('것', 'NNB'), ('을', 'NNG'), ('꺠', 'UN'), ('닫', 'VV'), ('지', 'NNG'), ('못하', 'VX'), ('고', 'NNG'), ('인간', 'NNG'), ('과', 'NNG'), ('대화', 'NNG'), ('를', 'UN'), ('계속', 'MAG'), ('하', 'NNG'), ('ㄹ', 'NNG'), ('수', 'NNG'), ('있', 'VA'), ('다면', 'NNG'), ('컴퓨터', 'NNG'), ('늘', 'VA'), ('지능', 'NNG'), ('적', 'NNG'), ('이', 'NNG'), ('ㄴ', 'NNG'), ('것', 'NNB'), ('으', 'UN'), ('간주', 'NNG'), ('되', 'VA'), ('ㄹ', 'NNG'), ('수', 'NNG'), ('있', 'VA'), ('다', 'NNG'), ('.', 'SF')]


In [ ]:
token_pos_cnt = Counter(tokens_tags).most_common()
token_pos_cnt

[(('있', 'VA'), 3),
 (('인간', 'NNG'), 2),
 (('이', 'NNG'), 2),
 (('컴퓨터', 'NNG'), 2),
 (('대화', 'NNG'), 2),
 (('하', 'NNG'), 2),
 (('고', 'NNG'), 2),
 (('것', 'NNB'), 2),
 (('ㄹ', 'NNG'), 2),
 (('수', 'NNG'), 2),
 (('오', 'VA'), 1),
 (('달', 'VV'), 1),
 (('을', 'NNG'), 1),
 (('꺠', 'UN'), 1),
 (('닫', 'VV'), 1),
 (('지', 'NNG'), 1),
 (('못하', 'VX'), 1),
 (('과', 'NNG'), 1),
 (('를', 'UN'), 1),
 (('계속', 'MAG'), 1),
 (('다면', 'NNG'), 1),
 (('늘', 'VA'), 1),
 (('지능', 'NNG'), 1),
 (('적', 'NNG'), 1),
 (('ㄴ', 'NNG'), 1),
 (('으', 'UN'), 1),
 (('간주', 'NNG'), 1),
 (('되', 'VA'), 1),
 (('다', 'NNG'), 1),
 (('.', 'SF'), 1)]